#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [69]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [70]:
!pip install wget
#перед сдачей это закомментить

In [71]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [185]:
#%%write_and_run task_1.py


import numpy as np
import pandas as pd

category_translation = {
    'portateis_cozinha_e_preparadores_de_alimentos': 'portable kitchen and food preparers',
    'pc_gamer': 'PC Gamer'
}

translation['product_category_name'] = translation['product_category_name'].replace(category_translation)
def task_1(translation, items, products):
    merged_data = pd.merge(items, products, on='product_id')

    merged_data = pd.merge(merged_data, translation, on='product_category_name')

    result = merged_data.groupby('product_category_name_english').agg({'product_id': 'count', 'price': 'mean'}).reset_index()

    result.columns = ['category', 'products', 'price']
    return result

task_1(translation, items, products)


,category,products,price
0,agro_industry_and_commerce,212,342.124858
1,air_conditioning,297,185.269226
2,art,209,115.802105
3,arts_and_craftmanship,24,75.583750
4,audio,364,139.254121
...,...,...,...
66,stationery,2517,91.753369
67,tablets_printing_image,83,90.703735
68,telephony,4545,71.213978
69,toys,4117,117.548360


In [141]:
#Проверки

res = task_1(translation, items, products)
assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

IndexError: ignored

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [154]:
#%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd


def task_2(translation, products, items):
    products_translated = products.merge(translation, on='product_category_name', how='left')

    seller_categories = items.merge(products_translated, on='product_id', how='left')

    seller_main_category = seller_categories.groupby('seller_id')['product_category_name_english'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else '').reset_index()
    seller_main_category.rename(columns={'product_category_name_english': 'category'}, inplace=True)

    return seller_main_category

result = task_2(translation, products, items)

print(result.head())


                          seller_id          category
0  0015a82c2db000af6aaaf3ae2ecb0532  small_appliances
1  001cca7ae9ae17fb1caed9dfb1094831      garden_tools
2  001e6ad469a905060d959994f1b41e4f    sports_leisure
3  002100f778ceb8431b7a1020ff7ab48f   furniture_decor
4  003554e2dce176b5555353e4f3555ac8                  


In [155]:
# Проверки

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())

assert len(res) == 3035 #3095
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

65


AssertionError: ignored

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [172]:
#%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    # Объедините данные из таблиц orders, customers и items по необходимым ключам
    merged_data = pd.merge(orders, customers, how='inner', on='customer_id')
    merged_data = pd.merge(merged_data, items, how='inner', on='order_id')

    # Посчитайте общую сумму потраченных денег для каждого заказа
    merged_data['total_spent'] = merged_data['price'] + merged_data['freight_value']

    # Группируйте данные по штату и посчитайте общую сумму потраченных денег для каждого штата
    state_total_spent = merged_data.groupby('customer_state')['total_spent'].sum().reset_index()

    # Посчитайте общую сумму потраченных денег по всем штатам
    total_spent_all_states = state_total_spent['total_spent'].sum()

    # Рассчитайте долю потраченных денег по каждому штату
    state_total_spent['perc'] = state_total_spent['total_spent'] / total_spent_all_states

    return state_total_spent

# Проверка
res = task_3(orders, customers, items)
print(res)






   customer_state  total_spent      perc
0              AC     19669.70  0.001241
1              AL     96229.40  0.006074
2              AM     27835.73  0.001757
3              AP     16262.80  0.001026
4              BA    611506.67  0.038597
5              CE    275606.30  0.017395
6              DF    353229.44  0.022295
7              ES    324801.91  0.020501
8              GO    347706.93  0.021946
9              MA    151171.99  0.009542
10             MG   1856161.49  0.117156
11             MS    135956.67  0.008581
12             MT    186168.96  0.011750
13             PA    217647.11  0.013737
14             PB    140987.81  0.008899
15             PE    322237.69  0.020339
16             PI    108132.28  0.006825
17             PR    800935.44  0.050553
18             RJ   2129681.98  0.134419
19             RN    101895.08  0.006431
20             RO     57558.02  0.003633
21             RR     10064.62  0.000635
22             RS    885826.76  0.055911
23             S

In [190]:
import pandas as pd
import numpy as np
import zipfile
import wget

def test_task_3():
   # from task_3 import task_3
    url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
    filename = wget.download(url)

    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./')

    customers = pd.read_csv('olist_customers_dataset.csv')
    location = pd.read_csv('olist_geolocation_dataset.csv')
    items = pd.read_csv('olist_order_items_dataset.csv')
    payments = pd.read_csv('olist_order_payments_dataset.csv')
    reviews = pd.read_csv('olist_order_reviews_dataset.csv')
    orders = pd.read_csv('olist_orders_dataset.csv')
    products = pd.read_csv('olist_products_dataset.csv')
    translation = pd.read_csv('product_category_name_translation.csv')
    sellers = pd.read_csv('olist_sellers_dataset.csv')

    res = task_3(orders, customers, items)
    assert np.allclose(res.perc.sum(), 1)
    assert np.allclose(res[res.state == "RS"].perc.values[0], 0.055868056429816286)
    assert np.allclose(res.sort_values("perc", ascending=True).iloc[0, 1], 0.0005862290943146945)
    assert np.allclose(res.sort_values("perc", ascending=False).iloc[0, 1], 0.3741756035817322)
    assert len(res) == 27
test_task_3()

AssertionError: ignored

In [173]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1

assert res[res['customer_state'] == "RS"].perc.values[0] == 0.05591086460097634
print(res.sort_values("perc", ascending=True).iloc[0, 1]) #выводит 100064.62
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

10064.62


AssertionError: ignored

## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [94]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd


def task_4(items, orders, customers):
    # Объединим таблицы orders, items и customers, чтобы получить информацию о заказах, товарах и покупателях в одной таблице
    combined_data = orders.merge(items, on='order_id', how='inner')
    combined_data = combined_data.merge(customers, on='customer_id', how='inner')

    # Вычислим средний чек покупки (с разбивкой на стоимость товаров и стоимость доставки)
    average_order_price = combined_data.groupby('order_id')[['price', 'freight_value']].sum().mean()

    # Вычислим среднее число товаров в заказе
    average_items_per_order = combined_data.groupby('order_id')['product_id'].count().mean()

    # Вычислим среднее число покупок на пользователя
    average_orders_per_customer = combined_data.groupby('customer_id')['order_id'].nunique().mean()

    return (
        average_order_price['price'],
        average_order_price['freight_value'],
        average_items_per_order,
        average_orders_per_customer
    )

# Применяем функцию к данным и сохраняем результат
result = task_4(items, orders, customers)

# Выводим результат
print(result)


(137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0)


In [97]:
# Проверка

res = task_4(items, orders, customers)
assert np.allclose(res, (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0)) #МЕНЯТЬ В ГИТ

In [250]:
import pandas as pd
import numpy as np
import zipfile
import wget

def test_task_4():
    #from task_4 import task_4
    url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
    filename = wget.download(url)

    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./')

    customers = pd.read_csv('olist_customers_dataset.csv')
    location = pd.read_csv('olist_geolocation_dataset.csv')
    items = pd.read_csv('olist_order_items_dataset.csv')
    payments = pd.read_csv('olist_order_payments_dataset.csv')
    reviews = pd.read_csv('olist_order_reviews_dataset.csv')
    orders = pd.read_csv('olist_orders_dataset.csv')
    products = pd.read_csv('olist_products_dataset.csv')
    translation = pd.read_csv('product_category_name_translation.csv')
    sellers = pd.read_csv('olist_sellers_dataset.csv')

    res = task_4(items, orders, customers)
    assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0)
test_task_4()

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [248]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime


def task_5(reviews):

    reviews['date'] = pd.to_datetime(reviews['review_creation_date'])

    start_date ='2017.04.01'
    end_date = '2018.05.01'
    reviews_f = reviews[(reviews['date'] >=  start_date) & (reviews['date'] <  end_date)]
    reviews_f['date'] = reviews_f['date'].dt.strftime("%Y-%m-%d")

    csat_by_day = reviews_f.groupby(['date'], as_index = False).mean()
    csat_by_day.columns = ['date', 'csat']

    return csat_by_day



In [249]:
import pandas as pd
import numpy as np
import zipfile
import wget

def test_task_5():
    #from task_5 import task_5
    url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
    filename = wget.download(url)

    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./')

    customers = pd.read_csv('olist_customers_dataset.csv')
    location = pd.read_csv('olist_geolocation_dataset.csv')
    items = pd.read_csv('olist_order_items_dataset.csv')
    payments = pd.read_csv('olist_order_payments_dataset.csv')
    reviews = pd.read_csv('olist_order_reviews_dataset.csv')
    orders = pd.read_csv('olist_orders_dataset.csv')
    products = pd.read_csv('olist_products_dataset.csv')
    translation = pd.read_csv('product_category_name_translation.csv')
    sellers = pd.read_csv('olist_sellers_dataset.csv')

    res = task_5(reviews)
    assert res.date.min() == '2017-04-01'
    assert res.date.max() == '2018-04-30'
    assert np.allclose(res.csat.sum(), 1551.8881071384853)
    assert np.allclose(res[res.date == '2017-07-11'].csat.values[0], 4.291390728476821)
    assert np.allclose(res[res.date == '2018-02-09'].csat.values[0], 3.992156862745098)
    assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'
test_task_5()

<ipython-input-248-ef33b0495c52>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_f['date'] = reviews_f['date'].dt.strftime("%Y-%m-%d")
<ipython-input-248-ef33b0495c52>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  csat_by_day = reviews_f.groupby(['date'], as_index = False).mean()


## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [134]:
%%write_and_run task_6.py

import numpy as np
import pandas as pd
from dateutil.parser import parse

def task_6(reviews):
    # Convert review dates to datetime objects
    reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
    reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

    # Calculate the difference in days between review creation and answer
    reviews['days_to_answer'] = (reviews['review_answer_timestamp'] - reviews['review_creation_date']).dt.days

    # Group by 'days_to_answer' and calculate the mean csat and the count of orders
    result = reviews.groupby('days_to_answer').agg({'review_score': 'mean', 'order_id': 'count'}).reset_index()
    result.rename(columns={'review_score': 'csat', 'order_id': 'orders'}, inplace=True)

    # Sort the result by 'days_to_answer' in ascending order
    result = result.sort_values(by='days_to_answer')

    return result

# Example usage
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
res = task_6(reviews)
print(res)

     days_to_answer      csat  orders
0                 0  3.886663   24361
1                 1  4.235586   30995
2                 2  4.048685   15898
3                 3  4.194567   14062
4                 4  4.042382    4601
..              ...       ...     ...
209             446  5.000000       1
210             471  4.000000       1
211             508  5.000000       1
212             512  5.000000       1
213             518  1.000000       1

[214 rows x 3 columns]


In [135]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224

#assert np.allclose(res.days.values == np.sort(res.days.values))
assert np.allclose(res['days_to_answer'].values, np.sort(res['days_to_answer'].values))

assert len(res) == 214
assert res['days_to_answer'].min() == 0                                                   #МЕНЯТЬ В ГИТ
assert res['days_to_answer'].max() == 518
assert res[res['days_to_answer'] == 233].csat.values[0] == 3.0
assert res[res['days_to_answer'] == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [174]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):

    orders['order_delivered_customer_date'].fillna('2999-12-31', inplace=True)

    return orders

In [175]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [183]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    # Merge relevant data
    merged_data = pd.merge(orders, items, on='order_id')
    merged_data = pd.merge(merged_data, sellers, on='seller_id')
    merged_data = pd.merge(merged_data, customers, on='customer_id')

    # Filter for delivered orders
    delivered_orders = merged_data[merged_data['order_status'] == 'delivered']

    # Calculate the total number of delivered orders for each seller
    total_orders = delivered_orders.groupby('seller_id')['order_id'].count()

    # Calculate the number of orders sent to a different state for each seller
    diff_region_orders = delivered_orders[delivered_orders['customer_state'] != delivered_orders['seller_state']]
    diff_region_counts = diff_region_orders.groupby('seller_id')['order_id'].count()

    # Calculate the percentage of orders sent to different regions for each seller as a decimal
    percentage_diff_region = (diff_region_counts / total_orders)

    # Sort the sellers by the percentage in descending order
    sorted_sellers = percentage_diff_region.sort_values(ascending=False)

    # Select the top 10 sellers
    top_10_sellers = sorted_sellers.head(10)

    # Create the resulting DataFrame with decimal 'share' values
    result = pd.DataFrame({'seller_id': top_10_sellers.index, 'share': top_10_sellers.values})

    return result

task_8(orders, items, sellers, customers)

,seller_id,share
0,0015a82c2db000af6aaaf3ae2ecb0532,1.0
1,c878515173c84a492ebe50d6b493230b,1.0
2,52f6f9f13ba19504c02781132a049e75,1.0
3,53088c75f948b48a7ece173ac17f0c12,1.0
4,53336f97ddbe53e2a617d07b17447b95,1.0
5,533d46c1b06970e99f09523a61df4972,1.0
6,c9d2ba8dde44a275ae06475ed1d2fafc,1.0
7,c9a06ece156bb057372c68718ec8909b,1.0
8,539ed9e19811677513447759be5eccfc,1.0
9,c97aa4ee7420f937da13b7f9e2228b99,1.0


In [184]:
# Проверки

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
print(res.share.values[0])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716

1.0


AssertionError: ignored